In [3]:
import requests
import json
import psycopg2
import os
from bs4 import BeautifulSoup

In [5]:
hostname = os.environ['CAPSTONE_DB_HOST']
dbname = os.environ['CAPSTONE_DB_DBNAME']
username = os.environ['CAPSTONE_DB_USERNAME']
password = os.environ['CAPSTONE_DB_PASSWORD']

In [6]:
response = requests.get('https://api.scryfall.com/cards/search?q=format:modern')
response.status_code

200

In [7]:
json_response = json.loads(response.text)

json_response.keys()

dict_keys(['object', 'total_cards', 'has_more', 'next_page', 'data'])

In [ ]:
json_re

In [ ]:
sample_card = json_response['data'][1]
sample_card

In [ ]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))

In [ ]:
cursor = conn.cursor()

In [ ]:
wanted_keys = ['cmc', 'collector_number', 'color_identity', 'id', 'mana_cost', 'name', 'set', 'type_line', 
               'object', 'multiverse_ids', 'mtgo_id', 'oracle_text', 'artist', 'set_name']

In [ ]:
filtered_card = {wanted_key: sample_card[wanted_key] for wanted_key in wanted_keys}

In [ ]:
data = [sample_card[wanted_key] for wanted_key in wanted_keys]

In [ ]:
template = ', '.join(['%s'] * len(data))
template

In [ ]:
query = '''INSERT INTO cards 
(cmc, collector_number, color_identity, scryfall_id, mana_cost, name, 
set, type_line, object, multiverse_ids, mtgo_id, oracle_text, artist, set_name)
VALUES ({})'''.format(template)


In [ ]:
query.split('\n')

In [ ]:
data[0]

In [ ]:
cursor.execute(query=query, vars=data)

In [ ]:
conn.commit()

In [ ]:
def upload_card(card, cursor, verbose=False):
    '''
    Uploads a card to the PSQL databse 'cards'.
    
    INPUT:
        - card: dictionary, magic the gathering card taken from scryfall.com
        - cursor: psycopg2 cursor object, used to insert card into db
        
    OUTPUT:
        - success: bool, True if no problems were encountered when inserting into db
    '''
    print(card['name'])
    # the wanted data fields from the card
    wanted_keys = ['cmc', 'collector_number', 'color_identity', 'id', 'mana_cost', 'name', 'set', 'type_line', 
                   'object', 'oracle_text', 'set_name']
    
    # filtering out unwanted keys
    data = [card[wanted_key] for wanted_key in wanted_keys]
    
    # creating the template to insert into sql
    template = ', '.join(['%s'] * len(data))
    
    # create sql query
    query = '''INSERT INTO cards 
                (cmc, collector_number, color_identity, scryfall_id, mana_cost, name, 
                set, type_line, object, oracle_text, set_name)
                VALUES ({})'''.format(template)
    
    # add data into the db
#     try:
    cursor.execute(query=query, vars=data)
#     except:
#         if verbose: print('could not insert card into db')
#         return False
    
    return True

In [11]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))

In [12]:
cursor = conn.cursor()

In [ ]:
for card in json_response['data']:
    if 'card_faces' in card.keys():
        print(card['name'])

In [ ]:
card = (json_response['data'][173])
card['legalities']

legalities = [k for k, v in card['legalities'].items() if v == 'legal']

legalities

In [ ]:
front_face = card['card_faces'][0]
front_face

In [ ]:
def format_card(card):
    '''
    Formats cards in a db friendly manner. If the card is multifaced, the first face is used for missing fields
    
    INPUT:
        - card: dictionary, magic card from scryfall.com
        
    OUTPUT: 
        - formatted_card: list, attributes of each card in the following order:
                            [name, cmc, type_line, oracle_text, mana_cost, power, toughness, colors, 
                            color_identity, legalities, set_code, set_name, collector_number, scryfall_id]
    '''
    keys = card.keys()
    formatted_card = []
    if 'card_faces' in keys:
        # multifaced card
        front_face = card['card_faces'][0]
        front_keys = front_face.keys()
        
        formatted_card.append(card['name'])
        formatted_card.append(card['cmc'])
        formatted_card.append(front_face['type_line'])
        
        if 'oracle_text' in front_keys:
            formatted_card.append(front_face['oracle_text'])
        else:
            formatted_card.append(None)
        
        formatted_card.append(front_face['mana_cost'])
        
        if 'power' in front_keys:
            formatted_card.append(front_face['power'])
        else:
            formatted_card.append(None)
        
        if 'toughness' in front_keys:
            formatted_card.append(front_face['toughness'])
        else:
            formatted_card.append(None)
            
        if 'colors' in keys:
            formatted_card.append(card['colors'])
        else:
            formatted_card.append(front_face['colors'])
            
        formatted_card.append(card['color_identity'])
        
        legalities = [k for k, v in card['legalities'].items() if v == 'legal']
        formatted_card.append(legalities)
        
        formatted_card.append(card['set'])
        formatted_card.append(card['set_name'])
        formatted_card.append(card['collector_number'])
        formatted_card.append(card['id'])
                                  
    else:
        # single faced card
        formatted_card.append(card['name'])
        formatted_card.append(card['cmc'])
        formatted_card.append(card['type_line'])
        
        if 'oracle_text' in keys: 
            formatted_card.append(card['oracle_text'])
        else:
            formatted_card.append(None)
            
        formatted_card.append(card['mana_cost'])
        
        if 'power' in keys:
            formatted_card.append(card['power'])
        else: 
            formatted_card.append(None)
        
        if 'toughness' in keys: 
            formatted_card.append(card['toughness'])
        else: 
            formatted_card.append(None)
        
        formatted_card.append(card['colors'])
        formatted_card.append(card['color_identity'])
        
        legalities = [k for k, v in card['legalities'].items() if v == 'legal']
        formatted_card.append(legalities)
        formatted_card.append(card['set'])
        formatted_card.append(card['set_name'])
        formatted_card.append(card['collector_number'])
        formatted_card.append(card['id'])
    
    return formatted_card

In [ ]:
cards = []
for idx, raw_card in enumerate(json_response['data']):
    card = format_card(raw_card)

    cards.append(card)
    
cards

In [ ]:
def upload_card(card, cursor, verbose=False):
    '''
    Uploads a card to the PSQL databse 'cards'.
    
    INPUT:
        - card: dictionary, magic the gathering card taken from scryfall.com
        - cursor: psycopg2 cursor object, used to insert card into db
        
    OUTPUT:
        - success: bool, True if no problems were encountered when inserting into db
    '''
    print(card['name'])
    # the wanted data fields from the card
    wanted_keys = ['cmc', 'collector_number', 'color_identity', 'id', 'mana_cost', 'name', 'set', 'type_line', 
                   'object', 'oracle_text', 'set_name']
    
    # filtering out unwanted keys
    data = [card[wanted_key] for wanted_key in wanted_keys]
    
    # creating the template to insert into sql
    template = ', '.join(['%s'] * len(data))
    
    # create sql query
    query = '''INSERT INTO cards 
                (cmc, collector_number, color_identity, scryfall_id, mana_cost, name, 
                set, type_line, object, oracle_text, set_name)
                VALUES ({})'''.format(template)
    
    # add data into the db
#     try:
    cursor.execute(query=query, vars=data)
#     except:
#         if verbose: print('could not insert card into db')
#         return False
    
    return True

In [21]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))

In [22]:
cursor = conn.cursor()

In [ ]:
cards[0]

In [ ]:
template = ', '.join(['%s'] * len(cards[0]))

template

In [ ]:
query = '''INSERT INTO cards (name, cmc, type_line, oracle_text, mana_cost, power, toughness, colors, 
                              color_identity, legalities, set_id, set_name, collector_number, scryfall_id)
           VALUES ({})'''.format(template)

query

In [ ]:
cursor.execute(query, cards[0])

In [ ]:
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM cards;')

In [ ]:
cursor.fetchall()

In [ ]:
def upload_card(card, cursor, verbose=False):
    
    template = ', '.join(['%s'] * len(card))
    
    query = '''INSERT INTO cards (name, cmc, type_line, oracle_text, mana_cost, power, toughness, colors, 
                              color_identity, legalities, set_id, set_name, collector_number, scryfall_id)
           VALUES ({})'''.format(template)

    try:
        cursor.execute(query, card)
        conn.commit()
        return True
    
    except psycopg2.IntegrityError:
        if verbose: print('duplicate card detected')
        return False


In [ ]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))
cursor = conn.cursor()
for idx, card in enumerate(cards):
    status = upload_card(card, cursor, verbose=True)
    if not status:
        conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))
        cursor = conn.cursor()

In [ ]:
url = 'https://api.scryfall.com/cards/search?q=format:modern'
response = requests.get(url)
json_response = json.loads(response.text)

In [ ]:
json_response['next_page']

In [ ]:
def get_modern_cards(verbose=False):
    '''Gets all modern legal cards from scryfall.com api'''
    
    conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))
    cursor = conn.cursor()

    url = 'https://api.scryfall.com/cards/search?q=format:modern'
    
    while True:
        response = requests.get(url)
        raw_cards = json.loads(response.text)
        
        for idx, raw_card in enumerate(raw_cards['data']):
            card = format_card(raw_card)
            status = upload_card(card, cursor, verbose=True)
            if not status:
                print('{}'.format(idx))
                conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))
                cursor = conn.cursor()
        if not raw_cards['has_more']:
            # no more cards to get
            if verbose: print('All done')
            break
        
        

In [1]:
json_response.keys()

NameError: name 'json_response' is not defined

In [9]:
from src.deck_scraping import get_cardstorm_id

In [28]:
get_cardstorm_id('Snapcaster Mage', cursor, verbose=True)

(20578, True)

In [59]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))
cursor = conn.cursor()

In [52]:
query = """SELECT cardstorm_id
           FROM cards
           WHERE name=%s"""

query

'SELECT cardstorm_id\n           FROM cards\n           WHERE name=%s'

In [53]:
cursor.execute(query, ["Gryff's Boon"])

In [54]:
cursor.fetchall()

[(10425,)]

In [29]:
from src.deck_scraping import *

In [32]:
front_page = BeautifulSoup(modern_front_page_request(page_number=0, verbose=True).text, 'html.parser')

requesting front page number 0


In [33]:
event_ids = get_event_ids(front_page, True)

    getting event ids from a front page


In [35]:
event_page = BeautifulSoup(event_request(event_ids[0], True).text, 'html.parser')

    event request for event id 18206


In [36]:
deck_ids = get_deck_ids(event_page, True)

In [37]:
raw_deck_list = deck_request(deck_ids[0], True)

        deck request for deck id 312786


In [38]:
deck_list = format_deck(raw_deck_list)

In [65]:
def make_user_card_counts_2(event_id, deck_id, deck_list, cursor, verbose=False):
    """
    Takes a deck_id and deck_list and returns user-card-count tuples.

    INPUT:
        - event_id: the unique identifier for the event this deck came from
        - deck_id: the unique identifier for the deck
        - deck_list: the deck list read from a file, formatted
        - cursor: psycopg2 cursor object, used to get cardstorm_id from db
        - verbose: bool, if true status messages will be printed
    OUTPUT:
        - user_card_count: a tuple containing the deck_id, card_name and
                           card_count for each card in the deck.
    """
    
    user_card_count = []
    for card_string in deck_list:
        parse_success, card_name, card_count = parse_card_string(card_string)
        cardstorm_id, cardstorm_success = get_cardstorm_id(card_name, cursor, verbose=verbose)
        if parse_success:
            user_card_count.append((int(event_id), int(deck_id), cardstorm_id, card_name, card_count))

    return user_card_count


In [66]:
user_card_counts = make_user_card_counts_2(event_ids[0], deck_ids[0], deck_list, cursor)

In [67]:
user_card_counts

[(18206, 312786, 8901, 'Dryad Arbor', 1),
 (18206, 312786, 10042, 'Gladecover Scout', 4),
 (18206, 312786, 11559, 'Kor Spiritdancer', 2),
 (18206, 312786, 20334, 'Silhana Ledgewalker', 2),
 (18206, 312786, 20527, 'Slippery Bogle', 4),
 (18206, 312786, 18590, 'Path to Exile', 2),
 (18206, 312786, 5606, 'Cartouche of Solidarity', 1),
 (18206, 312786, 8379, 'Daybreak Coronet', 4),
 (18206, 312786, 9242, 'Ethereal Armor', 4),
 (18206, 312786, 10425, "Gryff's Boon", 2),
 (18206, 312786, 10913, 'Hyena Umbra', 3),
 (18206, 312786, 17059, 'Leyline of Sanctity', 2),
 (18206, 312786, 19168, 'Rancor', 4),
 (18206, 312786, 20785, 'Spider Umbra', 3),
 (18206, 312786, 20827, 'Spirit Mantle', 2),
 (18206, 312786, 21859, 'Triclopean Sight', 2),
 (18206, 312786, 9700, 'Forest', 1),
 (18206, 312786, 10812, 'Horizon Canopy', 4),
 (18206, 312786, 18769, 'Plains', 2),
 (18206, 312786, 19218, 'Razorverge Thicket', 3),
 (18206, 312786, 21431, 'Temple Garden', 3),
 (18206, 312786, 22710, 'Windswept Heath', 3)

In [68]:
sum([x[-1] for x in user_card_counts])

60

In [70]:
get_cardstorm_id('Duskwatch Recruiter', cursor, True)

Duskwatch Recruiter


IndexError: list index out of range

In [71]:
url = 'https://api.scryfall.com/cards/named?exact=Duskwatch Recruiter'
response = requests.get(url)

response.status_code

200

In [73]:
duskwatch_recruiter = json.loads(response.text)

In [75]:
duskwatch_recruiter

{'artist': 'Craig J Spearing',
 'border_color': 'black',
 'card_faces': [{'colors': ['G'],
   'illustration_id': '7c7c6edf-7046-465d-93ff-1f13bbf3db73',
   'image_uris': {'art_crop': 'https://img.scryfall.com/cards/art_crop/en/soi/203a.jpg?1509844803',
    'border_crop': 'https://img.scryfall.com/cards/border_crop/en/soi/203a.jpg?1509844803',
    'large': 'https://img.scryfall.com/cards/large/en/soi/203a.jpg?1509844803',
    'normal': 'https://img.scryfall.com/cards/normal/en/soi/203a.jpg?1509844803',
    'png': 'https://img.scryfall.com/cards/png/en/soi/203a.png?1509844803',
    'small': 'https://img.scryfall.com/cards/small/en/soi/203a.jpg?1509844803'},
   'mana_cost': '{1}{G}',
   'name': 'Duskwatch Recruiter',
   'object': 'card_face',
   'oracle_text': '{2}{G}: Look at the top three cards of your library. You may reveal a creature card from among them and put it into your hand. Put the rest on the bottom of your library in any order.\nAt the beginning of each upkeep, if no spells 

In [76]:
url = 'https://api.scryfall.com/cards/named?exact=Dusk // Dawn'
response = requests.get(url)

response.status_code

200

In [77]:
dusk_dawn = json.loads(response.text)

dusk_dawn

{'artist': 'Noah Bradley',
 'border_color': 'black',
 'card_faces': [{'illustration_id': 'f3d63aed-2784-4ef5-9676-846b1e65e040',
   'mana_cost': '{2}{W}{W}',
   'name': 'Dusk',
   'object': 'card_face',
   'oracle_text': 'Destroy all creatures with power 3 or greater.',
   'type_line': 'Sorcery'},
  {'mana_cost': '{3}{W}{W}',
   'name': 'Dawn',
   'object': 'card_face',
   'oracle_text': 'Aftermath (Cast this spell only from your graveyard. Then exile it.)\nReturn all creature cards with power 2 or less from your graveyard to your hand.',
   'type_line': 'Sorcery'}],
 'cmc': 9.0,
 'collector_number': '210',
 'color_identity': ['W'],
 'colors': ['W'],
 'colorshifted': False,
 'digital': False,
 'edhrec_rank': 984,
 'eur': '0.83',
 'frame': '2015',
 'full_art': False,
 'futureshifted': False,
 'highres_image': True,
 'id': '937dbc51-b589-4237-9fce-ea5c757f7c48',
 'illustration_id': 'f3d63aed-2784-4ef5-9676-846b1e65e040',
 'image_uris': {'art_crop': 'https://img.scryfall.com/cards/art_cro

In [78]:
url = 'https://api.scryfall.com/cards/named?exact=Echo Mage'
response = requests.get(url)

response.status_code

200

In [79]:
echo_mage = json.loads(response.text)

echo_mage

{'artist': 'Matt Stewart',
 'border_color': 'black',
 'cmc': 3.0,
 'collector_number': '43',
 'color_identity': ['U'],
 'colors': ['U'],
 'colorshifted': False,
 'digital': False,
 'edhrec_rank': 2621,
 'eur': '0.28',
 'frame': '2003',
 'full_art': False,
 'futureshifted': False,
 'highres_image': True,
 'id': '8b76ba96-9630-44fb-849e-3c3848c03876',
 'illustration_id': 'e94e9826-ada9-4754-980a-4f1eb1c4f1e1',
 'image_uris': {'art_crop': 'https://img.scryfall.com/cards/art_crop/en/c13/43.jpg?1510051939',
  'border_crop': 'https://img.scryfall.com/cards/border_crop/en/c13/43.jpg?1510051939',
  'large': 'https://img.scryfall.com/cards/large/en/c13/43.jpg?1510051939',
  'normal': 'https://img.scryfall.com/cards/normal/en/c13/43.jpg?1510051939',
  'png': 'https://img.scryfall.com/cards/png/en/c13/43.png?1510051939',
  'small': 'https://img.scryfall.com/cards/small/en/c13/43.jpg?1510051939'},
 'layout': 'leveler',
 'legalities': {'1v1': 'legal',
  'commander': 'legal',
  'duel': 'legal',
  'f

In [91]:
def format_card(card):
    '''
    Formats cards in a db friendly manner. If the card is multifaced, the first face is used for missing fields
    
    INPUT:
        - card: dictionary, magic card from scryfall.com
        
    OUTPUT: 
        - formatted_card: list, attributes of each card in the following order:
                            [name, cmc, type_line, oracle_text, mana_cost, power, toughness, colors, 
                            color_identity, legalities, set_code, set_name, collector_number, scryfall_id]
    '''
    formatted_card = []
    layout = card['layout']
    keys = card.keys()
    
    if layout == 'normal':
        # normal layout, i.e, Lightning Bolt
        formatted_card.append(card['name'])
        formatted_card.append(card['cmc'])
        formatted_card.append(card['type_line'])
        
        if 'oracle_text' in keys: 
            formatted_card.append(card['oracle_text'])
        else:
            formatted_card.append(None)
            
        formatted_card.append(card['mana_cost'])
        
        if 'power' in keys:
            formatted_card.append(card['power'])
        else: 
            formatted_card.append(None)
        
        if 'toughness' in keys: 
            formatted_card.append(card['toughness'])
        else: 
            formatted_card.append(None)
        
        formatted_card.append(card['colors'])
        formatted_card.append(card['color_identity'])
        
        legalities = [k for k, v in card['legalities'].items() if v == 'legal']
        formatted_card.append(legalities)
        formatted_card.append(card['set'])
        formatted_card.append(card['set_name'])
        formatted_card.append(card['collector_number'])
        formatted_card.append(card['id'])
        
    elif layout == 'split':
        # split layout, i.e, (Dusk // Dawn), (Boom // Bust)
        side_a = card['card_faces'][0]
        side_b = card['card_faces'][1]
        
        formatted_card.append(card['name'])
        formatted_card.append(card['cmc'])
        formatted_card.append(' // '.join([side_a['type_line'], side_b['type_line']]))
        formatted_card.append(' // '.join([side_a['oracle_text'], side_b['oracle_text']]))
        formatted_card.append(card['mana_cost'])
        formatted_card.append(None)
        formatted_card.append(None)
        formatted_card.append(card['colors'])
        formatted_card.append(card['color_identity'])
        legalities = [k for k, v in card['legalities'].items() if v == 'legal']
        formatted_card.append(legalities)
        formatted_card.append(card['set'])
        formatted_card.append(card['set_name'])
        formatted_card.append(card['collector_number'])
        formatted_card.append(card['id'])
        
    elif layout == 'flip':
        # flip layout, i.e, Akki Lavarunner
        side_a = card['card_faces'][0]
        side_b = card['card_faces'][1]
        
        formatted_card.append(card['name'])
        formatted_card.append(card['cmc'])
        formatted_card.append(' // '.join([side_a['type_line'], side_b['type_line']]))
        formatted_card.append(' // '.join([side_a['oracle_text'], side_b['oracle_text']]))
        formatted_card.append(card['mana_cost'])
        formatted_card.append(side_a['power'])
        formatted_card.append(side_b['toughness'])
        formatted_card.append(card['colors'])
        formatted_card.append(card['color_identity'])
        legalities = [k for k, v in card['legalities'].items() if v == 'legal']
        formatted_card.append(legalities)
        formatted_card.append(card['set'])
        formatted_card.append(card['set_name'])
        formatted_card.append(card['collector_number'])
        formatted_card.append(card['id'])
        
    elif layout == 'transform':
        # transform layout, i.e, Delver of Secrets
        side_a = card['card_faces'][0]
        side_b = card['card_faces'][1]
        
        formatted_card.append(side_a['name'])
        formatted_card.append(card['cmc'])
        formatted_card.append(' // '.join([side_a['type_line'], side_b['type_line']]))
        formatted_card.append(' // '.join([side_a['oracle_text'], side_b['oracle_text']]))
        formatted_card.append(card['mana_cost'])
        formatted_card.append(side_a['power'])
        formatted_card.append(side_b['toughness'])
        formatted_card.append(side_a['colors'] + side_b['colors'])  
        formatted_card.append(card['color_identity'])
        legalities = [k for k, v in card['legalities'].items() if v == 'legal']
        formatted_card.append(legalities)
        formatted_card.append(card['set'])
        formatted_card.append(card['set_name'])
        formatted_card.append(card['collector_number'])
        formatted_card.append(card['id'])
        
    elif layout == 'meld':
        # meld layout, i.e, Bruna, the Fading Light
        # this layout turns out to be pretty much the same as a normal card
        formatted_card.append(card['name'])
        formatted_card.append(card['cmc'])
        formatted_card.append(card['type_line'])
        
        if 'oracle_text' in keys: 
            formatted_card.append(card['oracle_text'])
        else:
            formatted_card.append(None)
            
        formatted_card.append(card['mana_cost'])
        
        if 'power' in keys:
            formatted_card.append(card['power'])
        else: 
            formatted_card.append(None)
        
        if 'toughness' in keys: 
            formatted_card.append(card['toughness'])
        else: 
            formatted_card.append(None)
        
        formatted_card.append(card['colors'])
        formatted_card.append(card['color_identity'])
        
        legalities = [k for k, v in card['legalities'].items() if v == 'legal']
        formatted_card.append(legalities)
        formatted_card.append(card['set'])
        formatted_card.append(card['set_name'])
        formatted_card.append(card['collector_number'])
        formatted_card.append(card['id'])

    return formatted_card

In [116]:
url = 'https://api.scryfall.com/cards/named?exact=Beastbreaker of Bala Ged'
response = requests.get(url)

response.status_code

200

In [117]:
card = json.loads(response.content)

In [118]:
card

{'artist': 'Karl Kopinski',
 'border_color': 'black',
 'cmc': 2.0,
 'collector_number': '10',
 'color_identity': ['G'],
 'colors': ['G'],
 'colorshifted': False,
 'digital': False,
 'edhrec_rank': 12673,
 'eur': '0.04',
 'frame': '2015',
 'full_art': False,
 'futureshifted': False,
 'highres_image': True,
 'id': '9499b38f-5030-4348-a0ff-9002b0e46f2a',
 'illustration_id': '00b0b7fb-b87e-446c-be8f-fd6cd9f777cc',
 'image_uris': {'art_crop': 'https://img.scryfall.com/cards/art_crop/en/ddp/10.jpg?1509842264',
  'border_crop': 'https://img.scryfall.com/cards/border_crop/en/ddp/10.jpg?1509842264',
  'large': 'https://img.scryfall.com/cards/large/en/ddp/10.jpg?1509842264',
  'normal': 'https://img.scryfall.com/cards/normal/en/ddp/10.jpg?1509842264',
  'png': 'https://img.scryfall.com/cards/png/en/ddp/10.png?1509842264',
  'small': 'https://img.scryfall.com/cards/small/en/ddp/10.jpg?1509842264'},
 'layout': 'leveler',
 'legalities': {'1v1': 'legal',
  'commander': 'legal',
  'duel': 'legal',
  

In [115]:
from src.card_scraping import *

In [119]:
format_card_2(card)

formatting card Beastbreaker of Bala Ged


[]

In [120]:
' // '.join([None, 'abcd'])

[autoreload of src.card_scraping failed: Traceback (most recent call last):
  File "/Users/benjaminwalzer/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/Users/benjaminwalzer/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 369, in superreload
    module = reload(module)
  File "/Users/benjaminwalzer/anaconda2/envs/py36/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/benjaminwalzer/anaconda2/envs/py36/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._b

TypeError: sequence item 0: expected str instance, NoneType found

In [123]:
def get_cardstorm_id(card_name):
    cardstorm_id = ''
    for letter in card_name:
        cardstorm_id += str(ord(letter))
        
    return int(cardstorm_id)

In [127]:
get_cardstorm_id('snapcaster mage')

1151109711299971151161011143210997103101

In [130]:
'ABCD'.lower()

'abcd'

In [132]:
hostname = os.environ['CAPSTONE_DB_HOST']
dbname = os.environ['CAPSTONE_DB_DBNAME']
username = os.environ['CAPSTONE_DB_USERNAME']
password = os.environ['CAPSTONE_DB_PASSWORD']

In [133]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, hostname, username, password))

In [134]:
cursor = conn.cursor()

In [139]:
query = """SELECT cardstorm_id
           FROM cards
           WHERE name=%s"""
cursor.execute(query, ['delvr of secrets'])

In [141]:
bool(cursor.fetchall())

False

In [163]:
def get_cardstorm_id_2(card_name, cursor, verbose=False):
    '''
    Turns a card name into its' unique cardstorm id

    INPUT:
        - card_name: string, the name of a magic card
        - cursor: psycopg2 cursor object

    OUTPUT:
        - cardstorm_id: int, unique id for the card
        - success: bool, True if no exceptions were thrown
    '''

    query = """SELECT cardstorm_id
               FROM cards
               WHERE name=%s"""

    try:
        cursor.execute(query=query, vars=[card_name.lower()])

    except:
        if verbose: print('could not find cardstorm_id for {}'.format(card_name))
        return '', False
    if verbose: print(card_name)

    response = cursor.fetchone()[0]

    # if not response: # card was not found
    #     return None, False
    return response, True


In [164]:
get_cardstorm_id_2('Dryad Arbor', cursor, True)

Dryad Arbor


(2654, True)

In [146]:
card_name = 'Dryad Arbor'

In [147]:
card_name.lower()

'dryad arbor'

In [154]:
card_name = 'Dryad Arbor'

In [155]:
query = '''SELECT DISTINCT card_name, 
           FROM cards
           WHERE name=%s'''

cursor.execute(query, [card_name.lower()])
cursor.fetchall()

[(2654,)]

In [166]:
test = 'Rise / Fall'

a, b = test.split(' / ')

In [167]:
a

'Rise'

In [170]:
' // '.join(test.split(' / '))

'Rise // Fall'

In [171]:
test.replace(' / ', ' // ')

'Rise // Fall'

In [172]:
test

'Rise / Fall'

In [173]:
import re

In [197]:
re.search("(\d+)", '12 Lightning Bolt').group(1)

'12'

In [196]:
re.search(" (\D+)", '12 Lightning Bolt').group(1)

'Lightning Bolt'

In [181]:
re.search("", '12 Lightning Bolt').group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [188]:
re.findall('[0123456789]{0,3}', '12 Lightning Bolt')

['']

In [279]:
class CardDict():
    
    def __init__(self):
        self._dict = {}
    
    def __setitem__(self, key, val):
        self._dict[key] = val
        self._dict[val] = key
        
    def __getitem__(self, key):
        return self._dict[key]
    
    def keys(self):
        return self._dict.keys()
    
    def __contains__(self, key):
        return key in self._dict
    
    def get_cards(self):
        query = 'Select name, cardstorm_id FROM cards'
        
        try:
            cursor.execute(query)

            for name, cardstorm_id in cursor.fetchall():
                self[name] = cardstorm_id
        except:
            return False
        
        return True

In [283]:
card_dict = CardDict()

In [285]:
card_dict.get_cards()

True

In [286]:
card_dict[2259]

'delver of secrets'

In [322]:
dbname = os.environ['CAPSTONE_DB_DBNAME']
host = os.environ['CAPSTONE_DB_HOST']
username = os.environ['CAPSTONE_DB_USERNAME']
password = os.environ['CAPSTONE_DB_PASSWORD']

conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, host, username, password))

In [323]:
cursor = conn.cursor()

In [294]:
query = '''SELECT name, cardstorm_id
           FROM cardsasdf'''

cursor.execute(query)

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [277]:
card_dict = CardDict()
for name, cardstorm_id in cursor.fetchall():
    card_dict[name] = cardstorm_id

In [278]:
card_dict[2259]

'delver of secrets'

In [324]:
cursor.execute('SELECT DISTINCT deck_id FROM decks')

In [311]:
response = cursor.fetchall()

In [325]:
deck_ids = {_[0] for _ in cursor.fetchall()}

In [326]:
deck_ids

{307307,
 307308,
 307309,
 307310,
 307311,
 307312,
 307313,
 307314,
 307377,
 307577,
 307578,
 307579,
 307580,
 307581,
 307639,
 307640,
 307641,
 307642,
 307643,
 307644,
 307645,
 307646,
 307851,
 307852,
 307853,
 307854,
 307855,
 307856,
 307857,
 307858,
 307859,
 307860,
 307861,
 307862,
 307863,
 307864,
 307865,
 307866,
 307867,
 307868,
 307896,
 307897,
 307898,
 307899,
 307900,
 307901,
 307902,
 307903,
 307904,
 307905,
 307906,
 307907,
 307908,
 307909,
 307910,
 307911,
 307912,
 307913,
 307914,
 307915,
 307916,
 307917,
 307918,
 307919,
 307920,
 307921,
 307922,
 307923,
 307924,
 307925,
 307926,
 307927,
 307928,
 307929,
 307930,
 307931,
 307977,
 307978,
 307979,
 307980,
 307981,
 307982,
 307983,
 307984,
 308065,
 308066,
 308067,
 308068,
 308069,
 308070,
 308071,
 308072,
 308073,
 308074,
 308075,
 308076,
 308077,
 308078,
 308079,
 308080,
 308081,
 308082,
 308083,
 308084,
 308085,
 308086,
 308089,
 308090,
 308091,
 308092,
 308093,
 

In [320]:
1 in deck_ids

False